In [9]:
import cv2
import numpy as np
import os
from PIL import Image
import zipfile

# Parameter
video_path = r'C:\Users\jonni\OneDrive\Dokumente\GitHub\Chairlift_Gefahrenerkennung\5_Video\WhatsApp Video 2025-05-12 at 11.45.15.mp4'
output_dir = r'C:\Users\jonni\OneDrive\Dokumente\GitHub\Chairlift_Gefahrenerkennung\01_Data_Jon\Original'
os.makedirs(output_dir, exist_ok=True)

num_images = 50      # Gesamtanzahl gewünschter Bilder
num_regular = 20     # Davon regelmäßig verteilt
num_sturz = num_images - num_regular

# Video laden
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frames = []
frame_diffs = []

# Alle Frames und Differenzen berechnen
ret, prev_frame = cap.read()
if not ret:
    raise ValueError("Video konnte nicht geladen werden.")
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
frames.append(prev_frame)

for _ in range(1, frame_count):
    ret, curr_frame = cap.read()
    if not ret:
        break
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(curr_gray, prev_gray)
    diff_score = np.sum(diff)
    frame_diffs.append(diff_score)
    frames.append(curr_frame)
    prev_gray = curr_gray

cap.release()

# Regelmäßige Frames extrahieren
regular_indices = np.linspace(0, len(frames) - 1, num_regular, dtype=int)

# Bewegungsstärkste Frames extrahieren
if len(frame_diffs) < num_sturz:
    num_sturz = len(frame_diffs)
sturz_indices = np.argsort(frame_diffs)[-num_sturz:] + 1  # +1 weil Differenz ab Frame 1

# Kombinieren und sortieren, Duplikate entfernen
all_indices = np.unique(np.concatenate([regular_indices, sturz_indices]))
all_indices.sort()

extracted_files = []
for i, idx in enumerate(all_indices):
    frame = frames[idx]
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img)
    img_path = os.path.join(output_dir, f'frame_{i+1:02d}.jpg')
    img_pil.save(img_path)
    extracted_files.append(img_path)




'C:\\Users\\jonni\\OneDrive\\Dokumente\\GitHub\\Chairlift_Gefahrenerkennung\\01_Data_Jon\\Original\\sturz_frames.zip'